In [ ]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [ ]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- OR summary ---
    if "OR" in d.columns:
        or_mean = d["OR"].mean()
        or_std = d["OR"].std(ddof=1)
        df_out.loc[feature, "OR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std


    return df_out


def summarize_feature_rr(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- RR summary ---
    if "RR" in d.columns:
        or_mean = d["RR"].mean()
        or_std = d["RR"].std(ddof=1)
        df_out.loc[feature, "RR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std


    return df_out

features = ["Mono", "One", "Two", "Three", "Total"] 




from scipy.stats import chi2
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import statsmodels.api as sm



def hosmer_lemeshow(y_true, y_prob, g=10, eps=1e-12):

    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()

    if hl_df.shape[0] < g:
        return np.nan, np.nan

    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")
    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    if hl_table.shape[0] < 3:
        return np.nan, np.nan

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]

    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"]) ** 2) / (hl_table["expected"] + eps) +
        ((hl_table["observed_neg"] - hl_table["expected_neg"]) ** 2) / (hl_table["expected_neg"] + eps)
    ).sum()

    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def binarize_with_thresholds(s, low_thr, high_thr):
    """
    > high_thr -> 1
    <= low_thr -> 0
    entre medio -> NaN
    """
    return np.where(
        s > high_thr, 1,
        np.where(s <= low_thr, 0, np.nan)
    )




def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["OR"].mean()
    or_std  = d["OR"].std(ddof=1)
    df_out.loc[idx, "OR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minOR"] = d["2.5%"].min()
    df_out.loc[idx, "maxOR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Cross-sectional OR removing Slovakia

### Load data

In [ ]:
import time

t0 = time.perf_counter()


In [ ]:
data = pd.read_csv('data/BBAG-cross.csv')

In [ ]:
data = data[data.country != 'Slovakia'].reset_index(drop =  True)

In [ ]:
data

In [ ]:
vars_ = ['Mono', 'One', 'Two', 'Three', 'Total']

results_merge_df_all = data.copy()

rows = []



for boosts in range(10): 
    for i in vars_:
        count = 0

        a = int(np.floor(np.percentile(results_merge_df_all[i].dropna(), 5)))
        b = int(np.floor(np.percentile(results_merge_df_all[i].dropna(), 25)))
        c = int(np.ceil(np.percentile(results_merge_df_all[i].dropna(), 75)))
        d = int(np.ceil(np.percentile(results_merge_df_all[i].dropna(), 95)))

        low_thresholds = range(a, b)
        high_thresholds = range(c, d)

        for low_thr in low_thresholds:
            for high_thr in high_thresholds:

                tmp = results_merge_df_all.copy()

                tmp[i] = binarize_with_thresholds(tmp[i], low_thr, high_thr)

                tmp = tmp.dropna(subset=[i, "GAP_bin"]).reset_index(drop=True)



                y_ols = tmp["GAP_bin"]
                X_ols = tmp[[i]]

                test_size = 500 / tmp.shape[0]


                X_train, X_test, y_train, y_test = train_test_split(
                    X_ols, y_ols,
                    test_size=test_size,
                    #stratify=tmp["GAP_bin"],
                    random_state=boosts
                )

                X_train = X_train.copy()
                X_test = X_test.copy()
                X_train["intercept"] = 1
                X_test["intercept"] = 1


                model = sm.Logit(y_train, X_train).fit(disp=0)


                params = model.params
                conf = np.exp(model.conf_int())
                conf["OR"] = np.exp(params)
                conf["z"] = model.tvalues
                conf["P>|z|"] = model.pvalues
                conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

                df = conf.loc[i:i].copy()
                df["Feature"] = i

                df["low_thr"] = low_thr
                df["high_thr"] = high_thr
                df["boosts"] = boosts
                df["n_tmp"] = tmp.shape[0]  


                rows.append(df)

                count += 1

df_directions_odd = pd.concat(rows, axis=0).reset_index(drop=True) if len(rows) else pd.DataFrame()

In [ ]:
count*10

In [ ]:
df_directions_odd_f = pd.DataFrame()

for f in vars_:
    df_directions_odd_f = summarize_feature(df_directions_odd, f, df_directions_odd_f)

df_directions_odd_f.to_excel('Results/cross/cross_OR_-removing-slovakia-binarization.xlsx')
df_directions_odd_f

In [ ]:
dt = time.perf_counter() - t0
m, s = divmod(dt, 60)
print(f"Tiempo total: {int(m)} min {s:.1f} s")

## Cross-sectional OR with Slovakia

### Load data

In [ ]:
import time

t0 = time.perf_counter()


In [ ]:
data = pd.read_csv('data/BBAG-cross.csv')

In [ ]:
data

In [ ]:
vars_ = ['Mono', 'One', 'Two', 'Three', 'Total']

results_merge_df_all = data.copy()

rows = []

count = 0

for boosts in range(10): 
    for i in vars_:


        a = int(np.floor(np.percentile(results_merge_df_all[i].dropna(), 5)))
        b = int(np.floor(np.percentile(results_merge_df_all[i].dropna(), 25)))
        c = int(np.ceil(np.percentile(results_merge_df_all[i].dropna(), 75)))
        d = int(np.ceil(np.percentile(results_merge_df_all[i].dropna(), 95)))

        low_thresholds = range(a, b)
        high_thresholds = range(c, d)

        for low_thr in low_thresholds:
            for high_thr in high_thresholds:

                tmp = results_merge_df_all.copy()

                tmp[i] = binarize_with_thresholds(tmp[i], low_thr, high_thr)

                tmp = tmp.dropna(subset=[i, "GAP_bin"]).reset_index(drop=True)



                y_ols = tmp["GAP_bin"]
                X_ols = tmp[[i]]

                test_size = 500 / tmp.shape[0]


                X_train, X_test, y_train, y_test = train_test_split(
                    X_ols, y_ols,
                    test_size=test_size,
                    #stratify=tmp["GAP_bin"],
                    random_state=boosts
                )

                X_train = X_train.copy()
                X_test = X_test.copy()
                X_train["intercept"] = 1
                X_test["intercept"] = 1


                model = sm.Logit(y_train, X_train).fit(disp=0)


                params = model.params
                conf = np.exp(model.conf_int())
                conf["OR"] = np.exp(params)
                conf["z"] = model.tvalues
                conf["P>|z|"] = model.pvalues
                conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

                df = conf.loc[i:i].copy()
                df["Feature"] = i

                df["low_thr"] = low_thr
                df["high_thr"] = high_thr
                df["boosts"] = boosts
                df["n_tmp"] = tmp.shape[0]  


                rows.append(df)

                count += 1

df_directions_odd = pd.concat(rows, axis=0).reset_index(drop=True) if len(rows) else pd.DataFrame()

In [ ]:
df_directions_odd_f = pd.DataFrame()

for f in vars_:
    df_directions_odd_f = summarize_feature(df_directions_odd, f, df_directions_odd_f)

df_directions_odd_f.to_excel('Results/cross/cross_OR_-with-slovakia-binarization.xlsx')
df_directions_odd_f

In [ ]:
dt = time.perf_counter() - t0
m, s = divmod(dt, 60)
print(f"Tiempo total: {int(m)} min {s:.1f} s")